### PDF'ten Yazı İşlemek

Pdf'ten yazıları elde edebilmek için öncelikle pdf2image kütüphanesi sayesinde pdfleri resim formatına dönüştürüyoruz. Kütüphane için terminalden ; "pip install pdf2image" komutu ile kütüphaneyi indiriyoruz. Ancak bu kütüphane Poppler olmadan çalışmadığı için "conda install -c conda-forge poppler" komutu ile onun kurulumunu da yapıyoruz.

In [1]:
from pdf2image import convert_from_path
import argparse
import os
import cv2 as cv
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

## Bilgi Elde Etmek için Görüntü Bölgelerini İşaretleme
Not: Bölgeleri işaretleyebilmek için görüntüleri iyileştirmeliyiz. (DPI ≥ 300, Çarpıklık, Keskinlik ve Parlaklık ayarlanmalıdır, Eşikleme gibi)

OCR ile görüntülerdeki yazıları seçebiliyoruz. Doğrudan OCR kullanarak elde edilen sonuçlarda satır sonları bazı bozukluklara sebep olabiliyor. Bu yüzden de öncelikle yazıları tespit edip etrafını işaretleyerek bu sorunun üstesinden gelebiliriz.

In [2]:
# Pdfleri görüntülere dönüştürelim
pdfs = r"Samplepdfs\1.pdf"
pages = convert_from_path(pdfs, 350, poppler_path = r"C:\Program Files\poppler-0.68.0\bin")

# Pdfte ilgilendiğimiz yerleri işaretleyelim
def Mark_roi(image_path):

    img = cv.imread(image_path)

    THRESHOLD_REGION_IGNORE = 80

    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    blur = cv.GaussianBlur(gray, (9, 9), 0)

    threshold = cv.adaptiveThreshold(blur, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY_INV, 11, 30)

    kernel = cv.getStructuringElement(cv.MORPH_RECT, (9, 9))
    dilate = cv.dilate(threshold, kernel, iterations = 4)

    cnts = cv.findContours(dilate, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]

    line_items_cordinates = []
    for c in cnts:
        area = cv.contourArea(c)
        x, y, w, h = cv.boundingRect(c)

        if w < THRESHOLD_REGION_IGNORE or h < THRESHOLD_REGION_IGNORE:
            continue

        img = cv.rectangle(img, (x, y), (x + w, y + h), color=(255, 0, 255), thickness=3)
        line_items_cordinates.append([(x, y), (x + w, y + h)])

    return img, line_items_cordinates

# Görüntüleri saklamak için klasör oluşturalım
if not os.path.exists('images/pdf2image'):
    os.makedirs('images/pdf2image')

if not os.path.exists('images/bounding_images'):
    os.makedirs('images/bounding_images')

# Birden fazla sayfadan oluşursa pdf sayfalarını tek tek çevirebilmek için for içine alalım
i = 1
j = 1

for page in pages:
    image_name = "Page_" + str(i) + ".jpg"
    folder = "./images/pdf2image/" + str(image_name)
    page.save(folder, "JPEG")
    print(f"{image_name} oluşturuldu")

    # Görüntüdeki textlerin sınırlarını çizelim
    bounding_images = "image_" + str(j) + ".jpg"
    pic = "./images/bounding_images/" + str(bounding_images)
    image, line_items_cordinates = Mark_roi(folder)
    a_i = Image.fromarray(image)
    a_i.save(pic, "JPEG")
    print(f"{bounding_images} text sınırlı görüntü oluşturuldu")
    i += 1
    j += 1

print("Başarıyla tamamlandı!")

Page_1.jpg oluşturuldu
image_1.jpg text sınırlı görüntü oluşturuldu
Başarıyla tamamlandı!


## Elde Edilen Görüntüden Çıktı Alma

In [3]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract'

# Çıktıyı yazabilmek için dosya oluşturalım
outfile = "out_text.txt"

f = open(outfile, "a")

for i in range(1, i+1):
    text = str(pytesseract.image_to_string(pic, config='--psm 6'))
    text = text.replace('-\n', '')
    
    f.write(text)

f.close()

print("Dosya başarıyla oluşturuldu!")

Dosya başarıyla oluşturuldu!
